In [2]:
import numpy as np
import pandas as pd
import json
import os 
import sys
import pyvo as vo

In [42]:

class ExoplanetKeplerFileBuilder():

    def __init__(self):
        self.files_dir = '/home/gitika/My_Home/My_Projects/Alien_Megastructures/Files/' 
        self.quarterly_files_dir = self.files_dir + 'Kepler_Quarterly_wget/'
        self.kep_conf_csv = 'kep_conf_names_2025.06.10_03.15.42.csv'
        self.exoplanet_file = self.files_dir + 'PlanetarySystemsTableShort_18_06_2025.csv'


    def TapQueryPyvo(self):
        """
        Access the Planetary system Table from NASA exoplant archive
        """
        tap_service = vo.dal.TAPService("https://exoplanetarchive.ipac.caltech.edu/TAP")
        ex_query = """SELECT pl_name,hostname,sy_pnum,discoverymethod,disc_year,
        disc_facility,soltype,pl_controv_flag,pl_refname,pl_orbper,pl_orbpererr1,
        pl_orbpererr2,pl_orbperlim,pl_orbsmax,pl_orbsmaxerr1,pl_orbsmaxerr2,
        pl_orbsmaxlim,pl_rade,pl_radeerr1,pl_radeerr2,pl_radelim,pl_radj,pl_radjerr1,
        pl_radjerr2,pl_radjlim,pl_masse,pl_masseerr1,pl_masseerr2,pl_masselim,pl_massj,
        pl_massjerr1,pl_massjerr2,pl_massjlim,pl_orbeccen,pl_orbeccenerr1,pl_orbeccenerr2,
        pl_orbeccenlim,pl_orbincl,pl_orbinclerr1,pl_orbinclerr2,pl_orbincllim,pl_imppar,
        pl_impparerr1,pl_impparerr2,pl_impparlim,pl_trandep,pl_trandeperr1,pl_trandeperr2,
        pl_trandeplim,pl_trandur,pl_trandurerr1,pl_trandurerr2,pl_trandurlim,
        sy_umag,sy_umagerr1,sy_umagerr2,sy_gmag,sy_gmagerr1,sy_gmagerr2,sy_rmag,sy_rmagerr1,
        sy_rmagerr2,sy_imag,sy_imagerr1,sy_imagerr2,sy_zmag,sy_zmagerr1,sy_zmagerr2,sy_tmag,
        sy_tmagerr1,sy_tmagerr2,sy_kepmag,sy_kepmagerr1,sy_kepmagerr2 
        FROM ps
          """
        result = tap_service.search(ex_query)
        print(result)
        table = result.to_table()
        table.write(self.files_dir+'PlanetarySystemsTableShort_18_06_2025.csv', format='ascii.csv', overwrite=True)
            
    
    def readKepQuartBatFile(self,qid='0'):
        """ 
        Reads a single .dat file of a Kepler quarter and returns KIC id of all sources
        Return: kepid --> [kplr000000000,...]
        """
        filename = 'Kepler_Q'+qid+'_wget.bat'
        file = np.loadtxt(self.quarterly_files_dir+filename,delimiter=' ',dtype=str)
        nobjs = file.shape[0]
        kepid = [file[i][2].split('-')[0].replace("'","") for i in range(0,nobjs,1)]
        print('filename, nobjs',filename,nobjs)
        return kepid
    
    def getallUniqueKICnames(self):
        """ 
        Using .dat files from Q0-Q17 of kepler, makes catalogue of unique sources observed
        Return: unique_obj_kep --> array containin KIC of unique sources
                size_unique_kic --> total number of unique KIC names
        
        """
        obj_all_quart = []
        for i in range(0,18,1):
            qid = str(i)
            kepid = self.readKepQuartBatFile(qid=qid)
            obj_all_quart.extend(kepid)
            
        obj_all_quart = np.array(obj_all_quart)
        unique_obj_kep = np.unique(obj_all_quart)  #kep id/KIC of all sources observed b/w Q0-Q17
        size_unique_kic = unique_obj_kep.shape
        return unique_obj_kep, size_unique_kic
            
    
    def creatKeplerSourceList_confirmed(self,show=False):
        # cols: kepid,koi_name,kepler_name,pl_name
        kep_conf_list = np.loadtxt(self.files_dir+self.kep_conf_csv,delimiter=',',dtype=str)
        
        for i in range(1,kep_conf_list.shape[0],1): 
            if len(kep_conf_list[i][0]) < 9:
                nadd = 9-len(kep_conf_list[i][0])
                add = '0'*nadd # number of zeros to add to make kep id a 9 digit number
                kepid_conf = 'kplr'+add+kep_conf_list[i][0]
            else:
                kepid_conf = 'kplr'+kep_conf_list[i][0]
            #print('kepid_conf',kepid_conf)
            kep_conf_list[i][0] = kepid_conf
        if show:
            print(kep_conf_list[:,2]) # the first row is header row and first column is kep id
        return kep_conf_list
    
    def filterUniqueExoplanet(self,show=False):
        """ 
        This function reads 'Exoplanet_list_PS_2025.06.05_01.20.34.csv' containing many properties of the stars
        and planets detected around them.
        Returns the filtered array by removing repeated pl_name and selecting by non zero period.
        In case period not available source is kept in the list
        
        """
        usecols = ['pl_name','pl_orbper','sy_pnum','disc_facility']
        #exoplanet_file='/home/gitika/My_Home/My_Projects/Alien_Megastructures/PlanetarySystemsTableShort_18_06_2025.csv'
        exoplanet_data = pd.read_csv(self.exoplanet_file, usecols=usecols)
        print('exoplanet_data.columns',exoplanet_data.columns)

        self.Nrows_exofile = exoplanet_data.shape[0]
        print('Total number of rows in the file is: ', self.Nrows_exofile)
        # since many rows are repeated, remove repeated rows and keep the rows which have period non zero
        # In cases where period is zero for all repeated rows target is still kept in the list
    
        result = []
    
        for pl_name, group in exoplanet_data.groupby('pl_name'):
            # Ensure numeric and clean pl_orbper column
            group['pl_orbper'] = pd.to_numeric(group['pl_orbper'], errors='coerce')
            
            # Filter valid (non-zero, non-NaN) rows
            valid_rows = group[group['pl_orbper'].notna() & (group['pl_orbper'] != 0)]
            
            if not valid_rows.empty:
                result.append(valid_rows.iloc[[0]])  # Keep just one valid row
            else:
                result.append(group.iloc[[0]])       # Keep one row anyway if all are bad
        
        # Combine into single DataFrame
        filtered_exoplanet_data = pd.concat(result, ignore_index=True)
        if show:    
            print(filtered_exoplanet_data)
            
        self.Nplanstot_conf = filtered_exoplanet_data.shape[0]
        print("\033[1;32m Total number of exoplanets detected is:  %s\033[0m" %self.Nplanstot_conf)
        return filtered_exoplanet_data
    
    
    def createExoplanetFile_Kepler(self,kep_conf_list='',file_save_as='test.csv', show=False):
        filtered_exoplanet_data = self.filterUniqueExoplanet()
        trim_exopla_data = filtered_exoplanet_data
        
        # find the index of kepler confirmed sources in the filtered sources (exhausting list of all exoplanets)
        idx = np.where(np.isin(trim_exopla_data['pl_name'],kep_conf_list[1:,2]))[0] # indices of confirmed Kep planets in the filtered data
        #print('idx, idx.shape',idx,idx.shape)
        
        
        idx_notfound = np.where(~np.isin(kep_conf_list[1:,2],trim_exopla_data['pl_name']))[0] 
        #print('idx_notfound,idx_notfound.shape,kep_conf_list[idx_notfound]',idx_notfound,idx_notfound.shape,kep_conf_list[idx_notfound])
        
        self.Nplanskep_conf = kep_conf_list[1:,0].shape[0]
        self.Nplanskep_found = idx.shape[0]
        self.Nplanskep_notfound = idx_notfound.shape[0]
        print("\n")
        print("\033[1;32m Number of confirmed kepler exoplanets is:  %s\033[0m" %self.Nplanskep_conf)
        print("\033[1;32m Number of confirmed kepler exoplanets found in the NASA planetary systems file is: %s\033[0m" %self.Nplanskep_found)
        print("\033[1;32m Number of confirmed kepler exoplanets notfound in the NASA planetary systems file is: %s\033[0m" %self.Nplanskep_notfound)

        # filter the full exoplanet list to contain only Kepler confirmed sources
        new_exoplanet_data = trim_exopla_data.iloc[idx]
        #print('new_exoplanet_data.shape',new_exoplanet_data.shape)
        #print("new_exoplanet_data['pl_name']",new_exoplanet_data['pl_name'].iloc[0])

        new_exoplanet_data = new_exoplanet_data.copy()
        # Add two new columns and initialize them to None for all rows
        new_exoplanet_data['kicid']=None
        new_exoplanet_data['koi_name']=None
        
        for i in range(0,len(new_exoplanet_data),1):
            # find the index of kepler name in the confirmed list to get KIC 
            idx_temp = np.where(np.isin(kep_conf_list,new_exoplanet_data['pl_name'].iloc[i]))[0][0]

            nprint = int(new_exoplanet_data.shape[0]/2.0)
            if (i%nprint == 0 and show==True):
                #print('The cross matched kepler names',new_exoplanet_data.iloc[i, new_exoplanet_data.columns.get_loc('pl_name')],kep_conf_list[idx_temp][0][2])
                print('Kepler names in the two lists (should match)',kep_conf_list[idx_temp],new_exoplanet_data['pl_name'].iloc[i])
                #print('idx_temp',idx_temp,kep_conf_list[idx_temp],new_exoplanet_data['pl_name'].iloc[i])
            
            new_exoplanet_data.iloc[i, new_exoplanet_data.columns.get_loc('kicid')]=kep_conf_list[idx_temp][0]
            new_exoplanet_data.iloc[i, new_exoplanet_data.columns.get_loc('koi_name')]=kep_conf_list[idx_temp][1]
            
        #print('new_exoplanet_data',new_exoplanet_data)
        
        kep_conf_list_unique_kic = np.unique(kep_conf_list[1:,0])
        Nkepplan_unique_kic = len(kep_conf_list_unique_kic)
        print("\033[1;32m Number of light curves to download for confirmed Kepler planets is: %s\033[0m"%Nkepplan_unique_kic)
        print("\033[1;32m These light curves upon phase folding can be used for more multiple planets\033[0m")
        #print('len(kep_conf_list_unique_kic),kep_conf_list_unique_kic',len(kep_conf_list_unique_kic),kep_conf_list_unique_kic)
    
        
        # Generate new data array of confirmed kepler sources with KIC, sys_nplan,KOI name, pl_name, orbital period 
        data = []
        for i in range(0,len(kep_conf_list_unique_kic),1):  
            idx_temp = np.where(np.isin(new_exoplanet_data['kicid'],kep_conf_list_unique_kic[i]))[0]
            #print('idx_temp2',idx_temp,new_exoplanet_data.iloc[idx_temp])
            
            kicid = kep_conf_list_unique_kic[i] #new_exoplanet_data['kicid'].iloc[idx_temp]
            if len(idx_temp)!=0:
                
                koi_name = new_exoplanet_data['koi_name'].iloc[idx_temp]
                kepler_name = new_exoplanet_data['pl_name'].iloc[idx_temp]
                periods = new_exoplanet_data['pl_orbper'].iloc[idx_temp]
                sy_pnum = len(list(kepler_name))#new_exoplanet_data['pl_orbper'].iloc[idx_temp]
                
            else:
                idx_temp99 = np.where(np.isin(kep_conf_list,kicid))[0]
                koi_name = kep_conf_list[idx_temp99][:,1]
                kepler_name = kep_conf_list[idx_temp99][:,2]
                periods = np.empty(kep_conf_list[idx_temp99].shape[0])
                sy_pnum = kep_conf_list[idx_temp99].shape[0]

            row = {
                'kicid': kicid,
                'sy_pnum': sy_pnum,
                'koi_name': list(koi_name),
                'kepler_name': list(kepler_name),  # list of 1 to 5 integers
                'pl_orbper_days': list(periods),
            }
            data.append(row)
            if show:
                print('kicid:', kicid, '| kepler_names:', list(kepler_name))
            
        
        # Convert to DataFrame
        df = pd.DataFrame(data)
        #print('df',df)
        
        # Convert list column to string so it can be saved to CSV
        #df['kepler_names'] = df['kepler_names'].apply(json.dumps)
        #df['Kepler_name'] = df['Kepler_name'].apply(str)
        
        # Save to CSV
        df.to_csv(self.files_dir+file_save_as, index=False)
        print('\033[1;39m The file of confirmed Kepler exoplanets is saved as:\033[0m')
        print('%s'%self.files_dir+file_save_as)
        
        
    def nonExoplanetList_Kepler(self,unique_obj_kep='',kep_conf_list=''):
        
        indices_to_delete = np.where(np.isin(unique_obj_kep,kep_conf_list[1:,0]))[0]
        ndelete = indices_to_delete.shape[0]
        print('number of confirmed sources deleted from full list' ,ndelete,indices_to_delete)
        new_non_exoplanet_list = np.delete(unique_obj_kep, indices_to_delete)
        n_nonplanets = new_non_exoplanet_list.shape[0]
        print('Number of sources without planet detection in Kepler',n_nonplanets)
        
        np.savetxt(self.files_dir+'kepler_kic_non_exoplanets.csv',new_non_exoplanet_list,delimiter=",", fmt='%s')
        return n_nonplanets

    def GetSinglePlanets_KeplerList(self,savefile='test'):
        usecols = ['kicid', 'sy_pnum','kepler_name','pl_orbper_days']
        kep_conf_plans = pd.read_csv(self.files_dir+'kepler_confirmed_exoplanlist_created.csv', usecols=usecols)
        indices_single_plan = np.where(kep_conf_plans['sy_pnum']==1)[0]
        #print('indices_single_plan',indices_single_plan)
        single_plan_list = kep_conf_plans.iloc[indices_single_plan]
        N_kep_single = single_plan_list.shape[0]
        print('\033[1;39m Number of single planet systems in Kepler: %s\033[0m'%N_kep_single)
        #print('single_plan_list,single_plan_list.shape',single_plan_list,single_plan_list.shape)

        
        savefile_path = self.files_dir+savefile
        single_plan_list.to_csv(self.files_dir+savefile, index=False)
        print('Single planet fils is saved in %s'%savefile_path)

    def GetKOITable_PlanParams(self,kep_conf_list=''):
        
        koi_file_name = "koi_cumulative_2025.06.28_01.24.15.csv"
        koi_table = pd.read_csv(self.files_dir+koi_file_name,comment='#')
        print('KOI Table Columns, len(koi_table)',koi_table.columns,len(koi_table))
        print('kep_conf_list',kep_conf_list)

        snr_can_cut = 50.0
        
        unique_kois = np.unique(koi_table['kepoi_name'])
        n_unq_kois = len(unique_kois)
        unique_kois_snr_cut = unique_kois[unique_kois['koi_model_snr']>snr_can_cut]
        n_unique_sel = len(unique_kois_snr_cut)
        print('n_unq_kois',n_unq_kois)
        print('\033[1;39m All the KOI names are unique in koi_cumulative file \033[0m')
        print('\033[1;39m Number of unique KOI with snr >%s is %s \033[0m'%(snr_can_cut,n_unique_sel))
        

        false_pos_kois = koi_table[koi_table['koi_disposition']=='FALSE POSITIVE']
        n_fp_kois = len(false_pos_kois)
        
        candidate_kois = koi_table[koi_table['koi_disposition']=='CANDIDATE']
        n_candidate_kois = len(candidate_kois)
        print('\033[1;39m Number of False positive in KOI Table \033[0m',n_fp_kois)
        print('\033[1;39m Number of candidates in KOI Table \033[0m',n_candidate_kois)
        
        
        candidate_kois_sel = candidate_kois[candidate_kois['koi_model_snr']>snr_can_cut]
        n_candidate_sel = len(candidate_kois_sel)
        #print('candidate_kois_sel',candidate_kois_sel)
        print('\033[1;39m Number of candidates in KOI Table with snr>%s is %s\033[0m'%(snr_can_cut,
                                                                                       n_candidate_sel))
        
        idx_temp = np.where(np.isin(koi_table['kepler_name'],kep_conf_list[1:,2]))[0]
        macthed_koi_conf_tars = koi_table.iloc[idx_temp]
        N_kep_conf = kep_conf_list[1:].shape[0]
        N_plan_koi_kepconf = len(macthed_koi_conf_tars)
        N_koi_with_kepnames = koi_table['kepler_name'].replace('', pd.NA).dropna().count()
        
        print('Number of sources in kepler confirmed planet list is: %s'%N_kep_conf)
        print('Number of confirmed sources found in KOI cumulative table is: %s'%N_plan_koi_kepconf)
        print('Number of sources found in KOI table with kepler name is %s'%N_koi_with_kepnames)
        
        #print('macthed_koi_conf_tars,len(macthed_koi_conf_tars)',macthed_koi_conf_tars,len(macthed_koi_conf_tars))

        sel_targ_list = np.array(['Kepler-731 b', 'Kepler-5 b', 'Kepler-723 b', 
                                     'Kepler-423 b', 'Kepler-71 b', 'Kepler-6 b'])

        idx_sel = np.where(np.isin(koi_table['kepler_name'],sel_targ_list))[0]
        matched_sel_conf_tars = koi_table.iloc[idx_sel]
        #print('matched_sel_conf_tars,len(matched_sel_conf_tars)',matched_sel_conf_tars,len(matched_sel_conf_tars))
        transit_snr_sel = matched_sel_conf_tars['koi_model_snr']
        
        print('transit_snr_sel',transit_snr_sel)
        print('6 sources of the paper exist in the macthed_koi_conf_tars list')

        snr_cut = 1000.0
        high_snr_tars = macthed_koi_conf_tars[macthed_koi_conf_tars['koi_model_snr']>=snr_cut]
        N_high_snr_tars = len(high_snr_tars)
        print('Number of high SNR(>%s) targets in koi tabel is: %s'%(snr_cut,N_high_snr_tars)) 
        
        macthed_koi_conf_tars_new = macthed_koi_conf_tars.copy()
        macthed_koi_conf_tars_new['kepid'] = macthed_koi_conf_tars_new['kepid'].astype('object')
        for i in range(0,len(macthed_koi_conf_tars_new),1): 
            kepid = str(macthed_koi_conf_tars_new['kepid'].iloc[i])
            #print('len(kepid)',len(kepid))
            if len(kepid) < 9:
                nadd = 9-len(kepid)
                add = '0'*nadd # number of zeros to add to make kep id a 9 digit number
                kepid_conf = 'kplr'+add+kepid
            else:
                kepid_conf = 'kplr'+kepid
            #print('kepid_conf',kepid_conf)
            macthed_koi_conf_tars_new.iloc[i, macthed_koi_conf_tars_new.columns.get_loc('kepid')]= kepid_conf
            #macthed_koi_conf_tars['kepid'][i] = kepid_conf
        print('updated kepid',macthed_koi_conf_tars_new['kepid'][0]) # the first row is header row and first column is kep id
        
        
        
        #savefile_path = self.files_dir+savefile
        #single_plan_list.to_csv(self.files_dir+savefile, index=False)
        #print('Single planet fils is saved in %s'%savefile_path)
        
        use_cols = ['kepid','koi_count','kepler_name','koi_period','koi_model_snr','koi_ror']
        save_array = macthed_koi_conf_tars_new[use_cols].to_numpy()
        #save_array.to_csv(self.files_dir+'High_SNR_KOI_Planet_List.csv', index=False)

        df_to_save = pd.DataFrame(save_array, columns=use_cols)
        df_to_save.to_csv(self.files_dir + 'koi_kepconf_planet_list_with_snr.csv', index=False)

        # snr_koi_tars = macthed_koi_conf_tars['koi_model_snr']
        # plan_star_radius_ratio = macthed_koi_conf_tars['koi_ror']
        # limb_dark_mod = macthed_koi_conf_tars['koi_limbdark_mod']
        # limb_dark_coeff1 = macthed_koi_conf_tars['koi_ldm_coeff1']
        # limb_dark_coeff2 = macthed_koi_conf_tars['koi_ldm_coeff2']
        # limb_dark_coeff3 = macthed_koi_conf_tars['koi_ldm_coeff3']
        # limb_dark_coeff4 = macthed_koi_conf_tars['koi_ldm_coeff4']
        # print('limb_dark_mod',limb_dark_mod)
        return

    
        

kepbuilder = ExoplanetKeplerFileBuilder()        

In [6]:
kep_conf_list = kepbuilder.creatKeplerSourceList_confirmed()

In [44]:
kepbuilder.GetKOITable_PlanParams(kep_conf_list=kep_conf_list);

KOI Table Columns, len(koi_table) Index(['rowid', 'kepid', 'kepoi_name', 'kepler_name', 'koi_disposition',
       'koi_vet_stat', 'koi_vet_date', 'koi_pdisposition', 'koi_score',
       'koi_fpflag_nt',
       ...
       'koi_dicco_mdec', 'koi_dicco_mdec_err', 'koi_dicco_msky',
       'koi_dicco_msky_err', 'koi_dikco_mra', 'koi_dikco_mra_err',
       'koi_dikco_mdec', 'koi_dikco_mdec_err', 'koi_dikco_msky',
       'koi_dikco_msky_err'],
      dtype='object', length=141) 9564
kep_conf_list [['kepid' 'koi_name' 'kepler_name' 'pl_name']
 ['kplr011446443' 'K00001.01' 'Kepler-1 b' 'TrES-2 b']
 ['kplr011904151' 'K00072.01' 'Kepler-10 b' 'Kepler-10 b']
 ...
 ['kplr011758544' 'K01883.01' 'Kepler-997 b' 'Kepler-997 b']
 ['kplr011413812' 'K01885.01' 'Kepler-998 b' 'Kepler-998 b']
 ['kplr009549648' 'K01886.01' 'Kepler-999 b' 'Kepler-999 b']]


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [13]:
# Exoplanet Table columns saved into 'PlanetarySystemsTableShort_18_06_2025.csv' file
kepbuilder.TapQueryPyvo()

<DALResultsTable length=38500>
  pl_name    hostname sy_pnum ... sy_kepmag sy_kepmagerr1 sy_kepmagerr2
                              ...    mag         mag           mag     
   object     object   int32  ...  float64     float64       float64   
----------- --------- ------- ... --------- ------------- -------------
 Kepler-6 b  Kepler-6       1 ...    13.303            --            --
 Kepler-6 b  Kepler-6       1 ...    13.303            --            --
 Kepler-6 b  Kepler-6       1 ...    13.303            --            --
 Kepler-6 b  Kepler-6       1 ...    13.303            --            --
 Kepler-6 b  Kepler-6       1 ...    13.303            --            --
 Kepler-6 b  Kepler-6       1 ...    13.303            --            --
 Kepler-6 b  Kepler-6       1 ...    13.303            --            --
 Kepler-6 b  Kepler-6       1 ...    13.303            --            --
 Kepler-6 b  Kepler-6       1 ...    13.303            --            --
        ...       ...     ... ...

In [23]:
unique_obj_kep, size_unique_kic = kepbuilder.getallUniqueKICnames()
Nunique_kic_kep = unique_obj_kep.shape[0]
print("\033[1;32m Number unique KIC in Kepler Quarterly Data: %s \033[0m" %Nunique_kic_kep)

filename, nobjs Kepler_Q0_wget.bat 105780
filename, nobjs Kepler_Q1_wget.bat 313444
filename, nobjs Kepler_Q2_wget.bat 335710
filename, nobjs Kepler_Q3_wget.bat 334302
filename, nobjs Kepler_Q4_wget.bat 342714
filename, nobjs Kepler_Q5_wget.bat 333692
filename, nobjs Kepler_Q6_wget.bat 333366
filename, nobjs Kepler_Q7_wget.bat 329716
filename, nobjs Kepler_Q8_wget.bat 338742
filename, nobjs Kepler_Q9_wget.bat 337718
filename, nobjs Kepler_Q10_wget.bat 337026
filename, nobjs Kepler_Q11_wget.bat 331360
filename, nobjs Kepler_Q12_wget.bat 336966
filename, nobjs Kepler_Q13_wget.bat 336494
filename, nobjs Kepler_Q14_wget.bat 337008
filename, nobjs Kepler_Q15_wget.bat 332530
filename, nobjs Kepler_Q16_wget.bat 338248
filename, nobjs Kepler_Q17_wget.bat 337198
 Number unique KIC in Kepler Quarterly Data: 208281 


In [49]:
# confirmed list of kepler targets with KIC
kep_conf_list = kepbuilder.creatKeplerSourceList_confirmed()

In [25]:
# create csv file for non-confirmed kepler exoplanets 
kepbuilder.nonExoplanetList_Kepler(unique_obj_kep=unique_obj_kep,kep_conf_list=kep_conf_list)

number of confirmed sources deleted from full list 1985 [     6     88    138 ... 205883 205936 206002]
Number of sources without planet detection in Kepler 206296


206296

In [51]:
# create csv file for confirmed kepler exoplanets 
kepbuilder.createExoplanetFile_Kepler(kep_conf_list=kep_conf_list,file_save_as='kepler_confirmed_exoplanlist_created.csv')

exoplanet_data.columns Index(['pl_name', 'sy_pnum', 'disc_facility', 'pl_orbper'], dtype='object')
Total number of rows in the file is:  38500
 Total number of exoplanets detected is:  5921


 Number of confirmed kepler exoplanets is:  2815
 Number of confirmed kepler exoplanets found in the NASA planetary systems file is: 2759
 Number of confirmed kepler exoplanets notfound in the NASA planetary systems file is: 56
 Number of light curves to download for confirmed Kepler planets is: 1985
 These light curves upon phase folding can be used for more multiple planets
 The file of confirmed Kepler exoplanets is saved as:
/home/gitika/My_Home/My_Projects/Alien_Megastructures/Files/kepler_confirmed_exoplanlist_created.csv


In [68]:
kepbuilder.GetSinglePlanets_KeplerList(savefile='kepler_single_exoplanlist_created.csv')

 Number of single planet systems in Kepler: 1462
Single planet fils is saved in /home/gitika/My_Home/My_Projects/Alien_Megastructures/Files/kepler_single_exoplanlist_created.csv
